In [39]:
#Dwave imports
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dimod
from __future__ import print_function

import dimod
import math
import sys
import copy

from dimod.generators.constraints import combinations
from hybrid.reference import KerberosSampler

#Impots for QUBO probelm
import numpy as np
from numpy import linalg as LA
import pandas as pd
import matplotlib as plt
import itertools as it
from itertools import product
from matplotlib import pyplot as plt
import math

In [98]:
layers = 2 #no of squares
x_card = 5 #x length
y_card = x_card #y length

In [99]:
A = np.zeros((layers,x_card,y_card), dtype = 'int8')

In [100]:
# dqm.set_quadratic(s_l1_i_j_d ,s_l2_i_j_d, 1) for all d

In [101]:
def get_label(layer, row, col, digit):
    """Returns a string of the cell coordinates and the cell value in a
    standard format.
    """
    return "s_{layer}_{row}_{col}_{digit}".format(**locals()) # format 1 -> 01 for lex ordering - google me

def get_label_int(layer1, layer2, row, col, d1, d2):
    if layer1 == layer2:
        return
    else:
        return f"s_{layer1}_{row}_{col}_{d1}, s_{layer2}_{row}_{col}_{d2}" # format 1 -> 01 for lex ordering - google me

In [102]:
def build_bqm(matrix):
    """Build BQM using Sudoku constraints"""
    # Set up
    l = matrix.shape[0]          # Number of rows/columns in sudoku
    x = matrix.shape[1]    # Number of rows/columns in sudoku subsquare
    y = matrix.shape[2]
    digits = range(1, x+1)

    bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.SPIN)
    dict = {}
    # Constraint: Each node can only select one digit
    for row in range(x):
        for col in range(y):
            for layer in range(l):
                node_digits = [get_label(layer, row, col, digit) for digit in digits]
                one_digit_bqm = combinations(node_digits, 1)
                #print(node_digits)
                bqm.update(one_digit_bqm)
                dict.update(one_digit_bqm.to_qubo()[0])
                
    # Constraint: Each row of nodes cannot have duplicate digits
    for row in range(x):
        for  layer in range(l):
            for digit in digits:
                row_nodes = [get_label(layer, row, col, digit) for col in range(y)]
                row_bqm = combinations(row_nodes, 1)
                #print(row_nodes)
                bqm.update(row_bqm)
                dict.update(row_bqm.to_qubo()[0])

    # Constraint: Each column of nodes cannot have duplicate digits
    for col in range(y):
        for layer in range(l):
            for digit in digits:
                col_nodes = [get_label(layer, row, col, digit) for row in range(x)]
                col_bqm = combinations(col_nodes, 1)
                #print(col_nodes)
                bqm.update(col_bqm)
                dict.update(col_bqm.to_qubo()[0])


    for row in range(x):
        for col in range(y):
            for layer1 in range(l):
                for layer2 in range(l):
                    if layer1 != layer2:
                        node_layers =[get_label_int(layer1, layer2, row, col, d1, d2) for (d1,d2) in it.product(digits, repeat = 2)]
                        #print(node_layers)
                        layer_bqm = combinations(node_layers, 1)
                        print(layer_bqm)
                    

                

    return bqm # , dict

In [103]:
bqm = build_bqm(A)

's_0_4_4_5, s_1_4_4_3', 's_0_4_4_5, s_1_4_4_1'): 2.0, ('s_0_4_4_5, s_1_4_4_3', 's_0_4_4_5, s_1_4_4_2'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_1, s_1_4_4_1'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_1, s_1_4_4_2'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_1, s_1_4_4_3'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_1, s_1_4_4_4'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_1, s_1_4_4_5'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_2, s_1_4_4_1'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_2, s_1_4_4_2'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_2, s_1_4_4_3'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_2, s_1_4_4_4'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_2, s_1_4_4_5'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_3, s_1_4_4_1'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_3, s_1_4_4_2'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_3, s_1_4_4_3'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_3, s_1_4_4_4'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_3, s_1_4_4_5'): 2.0, ('s_0_4_4_5, s_1_4_4_4', 's_0_4_4_4, s_1_4_4_1'): 2.0, ('s_0_4_4_5

In [109]:
hubo = dict

In [112]:
sol = dimod.ExactPolySolver().sample_hubo(hubo).to_pandas_dataframe()

ValueError: Maximum allowed dimension exceeded

In [84]:
#sol

In [94]:
solution = KerberosSampler().sample(bqm,
                                        max_iter=10,
                                        convergence=3,
                                        qpu_params={'label': 'Example - latin square'})
best_solution = solution.first.sample
solution_list = [k for k, v in best_solution.items() if v == 1]


solutions = []
for label in solution_list:
        label = label.split('_')[1:]
        #solutions.append(label.split('_')[1:])
solutions = [label.split('_')[1:] for label in solution_list]


simplified_coords = []
k_k = []
for i in range(x_card**2):
    simplified_coords.append((int(solutions[i][1]),int(solutions[i][2]) ,int(solutions[i][3]) ,int(solutions[i+x_card**2][3])))
    k_k.append((int(solutions[i][3]) ,int(solutions[i+x_card**2][3])))


In [96]:
k_k

[(3, 2),
 (4, 3),
 (5, 1),
 (2, 4),
 (1, 5),
 (1, 3),
 (5, 5),
 (2, 2),
 (3, 1),
 (4, 4),
 (2, 1),
 (1, 4),
 (3, 3),
 (4, 5),
 (5, 2),
 (5, 5),
 (3, 2),
 (4, 4),
 (1, 3),
 (2, 1),
 (4, 4),
 (2, 1),
 (1, 5),
 (5, 2),
 (3, 3)]

In [95]:
solution_list

['s_0_0_0_3',
 's_0_0_1_4',
 's_0_0_2_5',
 's_0_0_3_2',
 's_0_0_4_1',
 's_0_1_0_1',
 's_0_1_1_5',
 's_0_1_2_2',
 's_0_1_3_3',
 's_0_1_4_4',
 's_0_2_0_2',
 's_0_2_1_1',
 's_0_2_2_3',
 's_0_2_3_4',
 's_0_2_4_5',
 's_0_3_0_5',
 's_0_3_1_3',
 's_0_3_2_4',
 's_0_3_3_1',
 's_0_3_4_2',
 's_0_4_0_4',
 's_0_4_1_2',
 's_0_4_2_1',
 's_0_4_3_5',
 's_0_4_4_3',
 's_1_0_0_2',
 's_1_0_1_3',
 's_1_0_2_1',
 's_1_0_3_4',
 's_1_0_4_5',
 's_1_1_0_3',
 's_1_1_1_5',
 's_1_1_2_2',
 's_1_1_3_1',
 's_1_1_4_4',
 's_1_2_0_1',
 's_1_2_1_4',
 's_1_2_2_3',
 's_1_2_3_5',
 's_1_2_4_2',
 's_1_3_0_5',
 's_1_3_1_2',
 's_1_3_2_4',
 's_1_3_3_3',
 's_1_3_4_1',
 's_1_4_0_4',
 's_1_4_1_1',
 's_1_4_2_5',
 's_1_4_3_2',
 's_1_4_4_3']

In [ ]:
# Copyright 2019 D-Wave Systems Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import print_function

import dimod
import math
import sys
import copy

from dimod.generators.constraints import combinations
from hybrid.reference import KerberosSampler


def get_label(row, col, digit):
    """Returns a string of the cell coordinates and the cell value in a
    standard format.
    """
    return "{row},{col}_{digit}".format(**locals())

def get_matrix(filename):
    """Return a list of lists containing the content of the input text file.

    Note: each line of the text file corresponds to a list. Each item in
    the list is from splitting the line of text by the whitespace ' '.
    """
    with open(filename, "r") as f:
        content = f.readlines()

    lines = []
    for line in content:
        new_line = line.rstrip()    # Strip any whitespace after last value

        if new_line:
            new_line = list(map(int, new_line.split(' ')))
            lines.append(new_line)

    return lines

def is_correct(matrix):
    """Verify that the matrix satisfies the Sudoku constraints.

    Args:
      matrix(list of lists): list contains 'n' lists, where each of the 'n'
        lists contains 'n' digits.
    """
    n = len(matrix)        # Number of rows/columns
    m = int(math.sqrt(n))  # Number of subsquare rows/columns
    unique_digits = set(range(1, n+1))  # Digits in a solution

    # Verifying rows
    for row in matrix:
        if set(row) != unique_digits:
            print("Error in row: ", row)
            return False

    # Verifying columns
    for j in range(n):
        col = [matrix[i][j] for i in range(n)]
        if set(col) != unique_digits:
            print("Error in col: ", col)
            return False
    return True


    

def build_bqm(matrix):
    """Build BQM using Sudoku constraints"""
    # Set up
    n = len(matrix)          # Number of rows/columns in sudoku
    m = int(math.sqrt(n))    # Number of rows/columns in sudoku subsquare
    digits = range(1, n+1)

    bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, dimod.SPIN)

    # Constraint: Each node can only select one digit
    for row in range(n):
        for col in range(n):
            node_digits = [get_label(row, col, digit) for digit in digits]
            one_digit_bqm = combinations(node_digits, 1)
            #print(node_digits)
            bqm.update(one_digit_bqm)

    # Constraint: Each row of nodes cannot have duplicate digits
    for row in range(n):
        for digit in digits:
            row_nodes = [get_label(row, col, digit) for col in range(n)]
            row_bqm = combinations(row_nodes, 1)
            #print(row_nodes)
            bqm.update(row_bqm)

    # Constraint: Each column of nodes cannot have duplicate digits
    for col in range(n):
        for digit in digits:
            col_nodes = [get_label(row, col, digit) for row in range(n)]
            col_bqm = combinations(col_nodes, 1)
            #print(col_nodes)
            bqm.update(col_bqm)


    # Constraint: Fix known values
    for row, line in enumerate(matrix):
        for col, value in enumerate(line):
            if value > 0:
                # Recall that in the "Each node can only select one digit"
                # constraint, for a given cell at row r and column c, we
                # produced 'n' labels. Namely,
                # ["r,c_1", "r,c_2", ..., "r,c_(n-1)", "r,c_n"]
                #
                # Due to this same constraint, we can only select one of these
                # 'n' labels (achieved by 'generators.combinations(..)').
                #
                # The 1 below indicates that we are selecting the label
                # produced by 'get_label(row, col, value)'. All other labels
                # with the same 'row' and 'col' will be discouraged from being
                # selected.
                bqm.fix_variable(get_label(row, col, value), 1)

    return bqm

def solve_sudoku(bqm, matrix):
    """Solve BQM and return matrix with solution."""
    solution = KerberosSampler().sample(bqm,
                                        max_iter=10,
                                        convergence=3,
                                        qpu_params={'label': 'Example - Sudoku'})
    best_solution = solution.first.sample
    solution_list = [k for k, v in best_solution.items() if v == 1]

    result = copy.deepcopy(matrix)

    for label in solution_list:
        coord, digit = label.split('_')
        row, col = map(int, coord.split(','))

        if result[row][col] > 0:
            # the returned solution is not optimal and either tried to
            # overwrite one of the starting values, or returned more than
            # one value for the position. In either case the solution is
            # likely incorrect.
            continue

        result[row][col] = int(digit)

    return result

if __name__ == "__main__":
    # Read user input
    if len(sys.argv) > 1:
        filename = sys.argv[1]
    else:
        filename = "problem.txt"
        print("Warning: using default problem file, '{}'. Usage: python "
              "{} <sudoku filepath>".format(filename, sys.argv[0]))

    # Read sudoku problem as matrix
    matrix = get_matrix(filename)

    # Solve BQM and update matrix
    bqm = build_bqm(matrix)
    result = solve_sudoku(bqm, matrix)

    # Print solution
    for line in result:
        print(*line, sep=" ")   # Print list without commas or brackets

    # Verify
    if is_correct(result):
        print("The solution is correct")
    else:
        print("The solution is incorrect")
